# Imports

In [31]:
# parser
from bs4 import BeautifulSoup
# data
import pandas as pd
import numpy as np
# requests
import requests
import os
# random
import random

# 1. Codeup Blog Articles

In [32]:
# visually inspecting, in order to pull article titles it will be an h2
url = 'https://codeup.edu/blog/'

### Header Randomization

In [73]:
# 
user_agents = [
  "Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0",
  "Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0",
  "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
  'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36',
  'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
  'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36,gzip(gfe)'
  ]
random_user_agent = random.choice(user_agents)

headers = {'User-Agent': random_user_agent}

response = requests.get(url, headers=headers)

In [74]:
# response status code
print(response)

# response text
response.text

<Response [200]>


'<!DOCTYPE html>\n<html lang="en-US">\n<head>\n\t<meta charset="UTF-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<link rel="pingback" href="https://codeup.edu/xmlrpc.php" />\n\n\t<script type="text/javascript">\n\t\tdocument.documentElement.className = \'js\';\n\t</script>\n\t\n\t<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin /><script id="diviarea-loader">window.DiviPopupData=window.DiviAreaConfig={"zIndex":1000000,"animateSpeed":400,"triggerClassPrefix":"show-popup-","idAttrib":"data-popup","modalIndicatorClass":"is-modal","blockingIndicatorClass":"is-blocking","defaultShowCloseButton":true,"withCloseClass":"with-close","noCloseClass":"no-close","triggerCloseClass":"close","singletonClass":"single","darkModeClass":"dark","noShadowClass":"no-shadow","altCloseClass":"close-alt","popupSelector":".et_pb_section.popup","initializeOnEvent":"et_pb_after_init_modules","popupWrapperClass":"area-outer-wrap","fullHeightClass":"full-height","openPopupClass

In [75]:
# soup object initiated
soup = BeautifulSoup(response.text, 'html.parser')

In [133]:
# setting blog titles var
blog_titles = soup.select('h2')
list_blog = list(blog_titles)

In [172]:
# location of urls within soup object
blog_titles[1].a['href']

'https://codeup.edu/featured/women-in-tech-panelist-spotlight/'

In [196]:
import contextlib
# retrieving blog names as list
blog_dict = {}
for i,blog in enumerate(list_blog):
    # ignore IndexError for aesthetics
    with  contextlib.suppress(IndexError):
    # extracting titles of blogs
        t = blog_titles[i].text
        # list comprehension to retrieve urls
        urls = [title.a['href'] for title in blog_titles if title.a]
        url = urls[i]
        blog_dict.__setitem__(t,url)

# dict
blog_dict


{'Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa': 'https://codeup.edu/featured/apida-heritage-month/',
 'Women in tech: Panelist Spotlight – Magdalena Rahn': 'https://codeup.edu/featured/women-in-tech-panelist-spotlight/',
 'Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill': 'https://codeup.edu/featured/women-in-tech-rachel-robbins-mayhill/',
 'Women in Tech: Panelist Spotlight – Sarah Mellor': 'https://codeup.edu/codeup-news/women-in-tech-panelist-spotlight-sarah-mellor/',
 'Women in Tech: Panelist Spotlight – Madeleine Capper': 'https://codeup.edu/events/women-in-tech-madeleine/',
 'Black Excellence in Tech: Panelist Spotlight – Wilmarie De La Cruz Mejia': 'https://codeup.edu/codeup-news/panelist-spotlight-4/'}

In [211]:
# blog names can be zipped with blog content once defined
codeup_blog = pd.DataFrame([blog_dict]).T.reset_index().rename(columns={'index':'title',0:'url'})
codeup_blog

,title,url
0,Spotlight on APIDA Voices: Celebrating Heritag...,https://codeup.edu/featured/apida-heritage-month/
1,Women in tech: Panelist Spotlight – Magdalena ...,https://codeup.edu/featured/women-in-tech-pane...
2,Women in tech: Panelist Spotlight – Rachel Rob...,https://codeup.edu/featured/women-in-tech-rach...
3,Women in Tech: Panelist Spotlight – Sarah Mellor,https://codeup.edu/codeup-news/women-in-tech-p...
4,Women in Tech: Panelist Spotlight – Madeleine ...,https://codeup.edu/events/women-in-tech-madele...
5,Black Excellence in Tech: Panelist Spotlight –...,https://codeup.edu/codeup-news/panelist-spotli...


## Retrieving content of all urls

In [335]:
# compiling code for 1 article, to see if it will be the same for the others
url_apida = 'https://codeup.edu/featured/apida-heritage-month/'
response = requests.get(url_apida, headers= headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [336]:
content = soup.select('p', class_='entry-content')

In [340]:
content[0].text

'May 24, 2023 | Featured'

In [282]:
# now that it has been determined that the content of every blog post is within the same schema, run a for loop to save it all -- could probably be done in a function
read = []
for url in codeup_blog['url']:
    print(f'This is the URL USED: {url}')
    response = requests.get(url, headers= headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.select('p', class_='entry-content')
    print(f'{content}\n\n + + + + + + + + + + + + + + + + + +\n')
    for i,c in enumerate(content):
        word = content[i].text
        read.append(word)

# indexing the new list and allocating to blog content list
blog_content = []

This is the URL USED: https://codeup.edu/featured/apida-heritage-month/
[<p class="post-meta"><span class="published">May 24, 2023</span> | <a href="https://codeup.edu/category/featured/" rel="category tag">Featured</a></p>, <p>May is traditionally known as Asian American and Pacific Islander (AAPI) Heritage Month. This month we celebrate the history and contributions made possible by our AAPI friends, family, and community. We also examine our level of support and seek opportunities to better understand the AAPI community.</p>, <p><img alt="arbeenathapa" class="wp-image-20059 alignright" decoding="async" fetchpriority="high" height="269" sizes="(max-width: 202px) 100vw, 202px" src="https://codeup.edu/wp-content/uploads/2023/05/upload-value-225x300.jpeg" srcset="https://codeup.edu/wp-content/uploads/2023/05/upload-value-225x300.jpeg 225w, https://codeup.edu/wp-content/uploads/2023/05/upload-value.jpeg 240w" width="202"/></p>, <p>In an effort to address real concerns and experiences, we

In [356]:
# manually indexing blog contents
article1 = read[0:26]
article2 = read[38:45]
article3 = read[57:65]
article4 = read[77:83]
article5 = read[95:101]
article6 = read[113:121]

In [362]:
blog_content = [article1,article2,article3,article4,article5,article6]

In [377]:
codeup_blog['content'] = blog_content
codeup_blog.content = codeup_blog.content.astype(str)

# 2. News Articles

In [379]:
# compiling code for 1 article, to see if it will be the same for the others
url = 'https://inshorts.com/en/read'
response = requests.get(url, headers= headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [382]:
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><title>Short English &amp; Hindi News|Latest News, India News, Current Affairs|Inshorts</title><link href="https://ajax.googleapis.com" rel="dns-prefetch"/><link href="https://assets.inshorts.com" rel="dns-prefetch"/><link href="https://static.inshorts.com" rel="dns-prefetch"/><link href="https://cdn.ampproject.org" rel="dns-prefetch"/><meta content="#ffffff" name="theme-color"/><link href="/assets/images/favicon.png" rel="shortcut icon" type="image/x-icon"><meta content="text/html; charset=utf-8" http-equiv="Content-Type"><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><link href="/assets/images/logo_192.png" rel="icon" sizes="192x192"><link href="https://inshorts.com" rel="canonical">
<link href="https://inshorts.com" rel="alternate">
<meta content="read latest news with inshorts in less than 60 words related to business sports bollywood and technolog

In [380]:
content = soup.select('p', class_='entry-content')

In [381]:
content

[]